In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib widget
import numpy as np
import k3d
import matplotlib.pyplot as plt
from functools import reduce
from ipywidgets import *
from matplotlib.animation import FuncAnimation
import matplotlib.animation as ani
from sloppy.raytracing import *
from sloppy.abcd import *

In [3]:
def SixMirror(dx=27.77, dy=8.0, dz=16.685, d=4.750, dzF=1.5825, Rfast=25.0, eps=np.zeros((6,5))):

    p5 = np.array([0,0,0])
    p6 = np.array([dx, dy, dzF])
    p1 = np.array([0, dy, dzF])
    p2 = np.array([dx, 2*dy, 0])
    p3 = np.array([d, dy+d, dz])
    p4 = np.array([dx-d, dy-d, dz])
    
    ps = np.stack([p1,p2,p3,p4,p5,p6], axis=0)
    geom = geometry(ps)
    ns = geom['refl']
    ps = geom['mir']
    angles = geom['angles']
    #add perturbations
    ps += eps[:,:3]
    axx = geom['xin']
    axy = 0.5*(geom['yin']+geom['yout'])
    ns = [rot_vec(rot_vec(ns[i], axx[i], np.deg2rad(eps[i,3])), axy[i], np.deg2rad(eps[i,4])) for i, n in enumerate(ns)]
    
    hi = 12.7
    qi=7.75
    #reference plane is expected between first and last element!
    elements = [CurvedMirror(p=ps[0], n=ns[0], diam=qi, R=Rfast, curv='CC', thet=angles[0]),\
                Mirror(p=ps[1], n=ns[1], diam=qi),\
                Mirror(p=ps[2], n=ns[2], diam=hi),\
                Mirror(p=ps[3], n=ns[3], diam=hi),\
                Mirror(p=ps[4], n=ns[4], diam=qi),\
                CurvedMirror(p=ps[5], n=ns[5], diam=qi, R=Rfast, curv='CC', thet=angles[5])]
    
    M = geom['M']
    R = geom['R']
    abcd = []
    for i, el in enumerate(elements):
        d = np.linalg.norm(M[i-1,i])/2
        #Mrt = Prop(d)@Rot(R[i])@Mirror(roc[i], angles[i])@Prop(d)@Mrt
        abcd.extend([Prop(d), el, Rot(R[i]), Prop(d)])
        
    return elements, abcd, geom

In [4]:
stab = lambda m: abs(0.5*np.trace(m))<1
La = 27.77

scanrng = 1.
N = 300
Las = La + np.linspace(-scanrng, scanrng, N)

ms = np.zeros((N))
ws = np.zeros((N,2))
freqs = np.zeros((N,4))
for i, l in enumerate(Las):
    _, abcd, _ = SixMirror(dx=l)
    system = ABCDSystem(abcd)
    
    ms[i] = stab(system.abcd_rt)
    w = system.waist_at(0)
    ws[i,:] = np.sort(w)
    freqs[i,...] = np.concatenate(system.get_freqs())

In [5]:
g, ax = plt.subplots(ncols=2, figsize=(8,4))
ax[0].plot(Las, ws*1e3)
ax[0].set_ylabel('um')
ax[1].plot(Las, freqs*1e-6)
ax[1].set_ylabel('MHz')
ax[1].axhline(0, color='grey')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
def prop_ABCD(mu, M, Nrt=100):
    rs = np.empty((Nrt+1,4))
    r = mu.copy()
    rs[i,:] = r
    for i in range(Nrt):
        r = M@r
        rs[i+1] = r
    return rs

In [7]:
elements, abcd, geom = SixMirror(dx=27.2952)

x0 = 0.5*(elements[0].p + elements[-1].p)
n0 = norm(elements[0].p - elements[-1].p)
rs = ray_bundle(p=x0, n=n0, n_radii=5, n_angles=2, R=0.2)

screen = Screen(p=x0, n=-n0, diam=7.75)
elements.append(screen)

system = ABCDSystem(abcd)

In [8]:
k = 2*np.pi/system.wl
k

8055.365778435367

In [9]:
mu1, mu2 = system.q

a1, a2 = 1., 1.
a1*mu1 + a2*mu2

array([-0.0070929 +1.21263311j,  0.02244248+1.2074685j ,
        0.80183035+0.12002002j,  0.85136874+0.05250147j])

In [10]:
mu = mu1.real*0.05

In [11]:
hit_m = prop_ABCD(mu, system.abcd_rt, Nrt=500)

hit_idx = np.arange(hit_m.shape[0])
plt.figure()
line = plt.scatter(hit_m[:,0], hit_m[:,1], c=hit_idx, cmap='jet')
plt.title('Poincare hit pattern (screen)')
plt.xlabel('x [mm]')
plt.ylabel('y [mm]')
plt.colorbar()
plt.show()

UnboundLocalError: local variable 'i' referenced before assignment

In [12]:
r0 = np.array([0, mu[1], mu[0]]) + x0
r0

array([13.6476    ,  7.9701796 ,  1.61434858])

In [13]:
a, b = mu[2:]/k
s0 = np.array([np.sqrt(1-a**2-b**2), b, a])
print(np.linalg.norm(s0))
s0

0.9999999999999999


array([1.00000000e+00, 1.91296776e-06, 3.44486889e-06])

In [14]:
ray0 = np.stack([np.atleast_2d(r0), np.atleast_2d(s0)], axis=0)
#reig, allseq, alltols = find_eigenray_mpe_debug(elements, ray0, Niter=20)

In [15]:
traj0 = propagate_system(elements, ray0, Nrt=30, clip=True)

In [16]:
try:
    plot.close()
except:
    pass
plot = k3d.plot(camera_auto_fit=True, antialias=True)

for i, el in enumerate(elements):
    pel = el.plot(opacity=0.4)
    plot += pel

plot += k3d.vectors(origins=r0, vectors=s0, colors=[(0xff0000,)*2], head_size=2.)
plot_trajs(traj0, plot, shader='mesh', width=0.2, color=0x00ff00)
#for ts in trajs_pert_ts:
#    plot += k3d.line(ts, shader='mesh', width=0.2, color=0x00ff00)

plot.display()

C:\ProgramData\Anaconda3\lib\site-packages\traittypes\traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  np.dtype(self.dtype).name))


Output()

In [17]:
traj_hit = propagate_system_at(elements, ray0, which=6, Nrt=500)[:,0,:,:] #only positions
hit_scr = traj_hit[:,0,:]
hit_idx = np.arange(hit_scr.shape[0])

plt.figure()
line = plt.scatter(hit_scr[:,1], hit_scr[:,2], c=hit_idx, cmap='jet')
plt.title('Poincare hit pattern (screen)')
plt.xlabel('x [mm]')
plt.ylabel('y [mm]')
plt.colorbar()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:
fig, (ax, bx) = plt.subplots(ncols=2, figsize=(10,5), sharex=True, sharey=True)
linea = ax.scatter(hit_scr[:,1]-x0[1], hit_scr[:,2]-x0[2], c=hit_idx, cmap='jet')
#linea = ax.scatter(hit_scr[:,1], hit_scr[:,2], c=hit_idx, cmap='jet')
lineb = bx.scatter(hit_m[:,0], hit_m[:,1], c=hit_idx, cmap='jet')
plt.xlabel('x [mm]')
plt.ylabel('y [mm]')

#@interact(dl=(27.2, 27.9, 1e-2), ar = (0,0.1,1e-3), br = (0,0.1,1e-3), ap= (0,2*np.pi,1e-2), bp= (0,2*np.pi,1e-2), Nrt=(0,1000,100))
def update(dl=27.295, ar=0.01, br=0.0, ap=0., bp=0., Nrt=500, **kwargs):
        elements, abcd, _ = SixMirror(dx=dl)#27.2952+
        x0 = 0.5*(elements[0].p + elements[-1].p)
        n0 = norm(elements[0].p - elements[-1].p)
        screen = Screen(p=x0, n=-n0, diam=7.75)
        elements.append(screen)
        system = ABCDSystem(abcd)

        mu1, mu2 = system.q
        mu = np.real(ar*np.exp(1j*ap)*mu1 + br*np.exp(1j*bp)*mu2)
        r0 = np.array([0, mu[1], mu[0]]) + x0
        a, b = mu[2:]/k
        s0 = np.array([np.sqrt(1-a**2-b**2), b, a])
        ray0 = np.stack([np.atleast_2d(r0), np.atleast_2d(s0)], axis=0)
 
        traj_hit = propagate_system_at(elements, ray0, which=6, Nrt=Nrt)[:,0,:,:] #only positions
        hit_scr = traj_hit[:,0,:]-x0
        #print(hit_scr)
        hit_idx = np.arange(hit_scr.shape[0])
        linea.set_offsets(hit_scr[:,1:])
        linea.set_array(hit_idx)
        
        
        hit_m = prop_ABCD(mu, system.abcd_rt, Nrt=Nrt)
        lineb.set_offsets(hit_m[:,0:2])
        lineb.set_array(hit_idx)
        
        ax.relim()
        ax.autoscale_view()
        fig.canvas.draw_idle()
        
#interactive(update, dl=(27.2, 27.9, 1e-2), ar = (0,0.1,1e-3), br = (0,0.1,1e-3), ap= (0,2*np.pi,1e-2), bp= (0,2*np.pi,1e-2), Nrt=(0,1000,100))
interactive(update, dl=(27.75, 27.81, 1e-3), ar = (0,0.1,1e-3), br = (0,0.1,1e-3), ap= (0,2*np.pi,1e-2), bp= (0,2*np.pi,1e-2), Nrt=(0,1000,100))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=27.75, description='dl', max=27.81, min=27.75, step=0.001), FloatSlide…

In [18]:
epsr = 1e-1
epss = 1e-1
rguideray = np.array([[0,0,0], [0,epsr,0], [0,0,epsr], [0,0,0], [0,0,0]]) + x0
sguideray = norm(n0 + np.array([[0,0,0], [0,0,0], [0,0,0], [0,epss,0], [0,0,epss]]))
guideray = np.stack([np.atleast_2d(rguideray), np.atleast_2d(sguideray)], axis=0)

In [19]:
trajgr = propagate_system(elements, guideray, Nrt=1, clip=True)

In [20]:
try:
    plot.close()
except:
    pass
plot = k3d.plot(camera_auto_fit=True, antialias=True)

for i, el in enumerate(elements):
    pel = el.plot(opacity=0.4)
    plot += pel

plot += k3d.vectors(origins=r0, vectors=s0, colors=[(0xff0000,)*2], head_size=2.)
plot_trajs(trajgr, plot, shader='mesh', width=0.1, color=0x00ff00)
#for ts in trajs_pert_ts:
#    plot += k3d.line(ts, shader='mesh', width=0.2, color=0x00ff00)

plot.display()

C:\ProgramData\Anaconda3\lib\site-packages\traittypes\traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  np.dtype(self.dtype).name))


Output()

In [41]:
trajgr[0]

array([[[13.6476    ,  8.        ,  1.5825    ],
        [13.6476    ,  8.1       ,  1.5825    ],
        [13.6476    ,  8.        ,  1.6825    ],
        [13.6476    ,  8.        ,  1.5825    ],
        [13.6476    ,  8.        ,  1.5825    ]],

       [[-1.        ,  0.        ,  0.        ],
        [-1.        ,  0.        ,  0.        ],
        [-1.        ,  0.        ,  0.        ],
        [-0.99503719,  0.09950372,  0.        ],
        [-0.99503719,  0.        ,  0.09950372]]])

In [46]:
trajfd = trajgr[-1]-trajgr[0]#
trajfd

array([[[ 0.00000000e+00, -8.88178420e-16,  1.11022302e-15],
        [ 0.00000000e+00, -1.04551736e-01,  1.72451348e-02],
        [ 0.00000000e+00,  2.87637628e-03, -1.20902447e-01],
        [ 0.00000000e+00, -1.23030217e-01,  7.15772947e-02],
        [ 0.00000000e+00, -6.73642050e-02, -1.16710423e-01]],

       [[ 6.66133815e-16,  1.66533454e-16, -1.42247325e-15],
        [ 2.48385885e-03,  6.18652361e-02, -3.36784902e-02],
        [ 2.36128041e-03,  3.43405520e-02,  5.94786656e-02],
        [-4.95020561e-03, -1.00581297e-01, -4.90377581e-03],
        [-4.47321736e-03, -1.60313332e-02, -1.26372688e-01]]])

In [47]:
trajfd.shape

(2, 5, 3)

In [59]:
trajfd[0,1,1:]/epss

array([-1.04551736,  0.17245135])

In [60]:
trajfd[1,1,1:]/epss

array([ 0.61865236, -0.3367849 ])

In [62]:
Mfd = np.concatenate([trajfd[0,1:,1:]/epsr, trajfd[1,1:,1:]/epss], axis=1).T
Mfd

array([[-1.04551736,  0.02876376, -1.23030217, -0.67364205],
       [ 0.17245135, -1.20902447,  0.71577295, -1.16710423],
       [ 0.61865236,  0.34340552, -1.00581297, -0.16031333],
       [-0.3367849 ,  0.59478666, -0.04903776, -1.26372688]])

In [67]:
G = np.array([[0,0,1,0], [0,0,0,1], [-1,0,0,0], [0,-1,0,0]])
np.round(Mfd.T@G@Mfd, decimals=3)

array([[ 0.   , -0.681,  2.045, -0.027],
       [ 0.681, -0.   ,  0.027,  2.449],
       [-2.045, -0.027,  0.   , -1.442],
       [ 0.027, -2.449,  1.442,  0.   ]])

In [63]:
system.abcd_rt

array([[ 0.14502622, -0.01037232, -1.34862271, -0.7145201 ],
       [-0.20946217,  0.21525424,  0.79639434, -1.27106549],
       [ 0.59368839,  0.33434547, -0.2760867 , -0.04643176],
       [-0.34204905,  0.61239838,  0.15767888, -0.08258424]])

In [68]:
def get_ABCD_fd(elements, x0, n0, epsr = 1e-1, epss = 1e-1, Nrt=1):

    rguideray = np.array([[0,0,0], [0,epsr,0], [0,0,epsr], [0,0,0], [0,0,0]]) + x0
    sguideray = norm(n0 + np.array([[0,0,0], [0,0,0], [0,0,0], [0,epss,0], [0,0,epss]]))
    guideray = np.stack([np.atleast_2d(rguideray), np.atleast_2d(sguideray)], axis=0)
    trajgr = propagate_system(elements, guideray, Nrt=Nrt, clip=True)
    trajfd = trajgr[-1]-trajgr[0]
    Mfd = np.concatenate([trajfd[0,1:,1:]/epsr, trajfd[1,1:,1:]/epss], axis=1).T
    return Mfd

In [88]:
trajfd2 = trajgr[-1,:,1:,:]-(trajgr[-1,:,0,:])[:,None,:]
trajfd2

array([[[ 0.00000000e+00, -4.55173604e-03,  1.72451348e-02],
        [ 0.00000000e+00,  2.87637628e-03, -2.09024469e-02],
        [ 0.00000000e+00, -1.23030217e-01,  7.15772947e-02],
        [ 0.00000000e+00, -6.73642050e-02, -1.16710423e-01]],

       [[ 2.48385885e-03,  6.18652361e-02, -3.36784902e-02],
        [ 2.36128041e-03,  3.43405520e-02,  5.94786656e-02],
        [ 1.26041751e-05, -1.07757784e-03, -4.90377581e-03],
        [ 4.89592426e-04, -1.60313332e-02, -2.68689692e-02]]])

In [89]:
def get_ABCD_fd2(elements, x0, n0, epsr = 1e-1, epss = 1e-1, Nrt=1):
    rguideray = np.array([[0,0,0], [0,epsr,0], [0,0,epsr], [0,0,0], [0,0,0]]) + x0
    sguideray = norm(n0 + np.array([[0,0,0], [0,0,0], [0,0,0], [0,epss,0], [0,0,epss]]))
    guideray = np.stack([np.atleast_2d(rguideray), np.atleast_2d(sguideray)], axis=0)
    trajgr = propagate_system(elements, guideray, Nrt=Nrt, clip=True)
    
    trajfd = trajgr[-1,:,1:,:]-(trajgr[-1,:,0,:])[:,None,:]
    Mfd = np.concatenate([trajfd[0,:,1:]/epsr, trajfd[1,:,1:]/epss], axis=1).T
    return Mfd

In [97]:
epss = np.logspace(-5, -1, 50)

In [71]:
epss = np.linspace(1e-5, 1e-1, 50)

In [98]:
Ms = np.stack([get_ABCD_fd2(elements, x0, n0, epsr=e, epss=e) for e in epss], axis=0)

In [100]:
plt.figure()
plt.semilogx(epss[:,None], Ms.reshape((-1, 16)), marker='.')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [101]:
plt.figure()
plt.semilogx(epss[:,None], np.einsum("xji,jk,xkl->xil", Ms, G, Ms).reshape((-1, 16)), marker='.')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [102]:
abcd_fd = get_ABCD_fd2(elements, x0, n0, epsr=1e-3, epss=1e-3) 

In [104]:
abcd_fd

array([[-0.10628955, -0.01087795, -1.21831545, -0.69206554],
       [ 0.18844328, -0.24385796,  0.69248005, -1.17875513],
       [ 0.6165635 ,  0.34009922, -0.10598962, -0.18835962],
       [-0.34003321,  0.59619397,  0.01069634, -0.24451704]])

In [106]:
np.round(abcd_fd.T@G@abcd_fd, decimals=3)

array([[ 0.   , -0.   ,  1.   , -0.   ],
       [ 0.   , -0.   ,  0.   ,  1.   ],
       [-1.   , -0.   ,  0.   , -0.001],
       [ 0.   , -1.   ,  0.001, -0.   ]])

In [107]:
hit_m = prop_ABCD(mu, system.abcd_rt, Nrt=500)
hit_fd = prop_ABCD(mu, abcd_fd, Nrt=500)
hit_idx = np.arange(hit_m.shape[0])
f, ax = plt.subplots(figsize=(9,4), ncols=2)
line = ax[0].scatter(hit_m[:,0], hit_m[:,1], c=hit_idx, cmap='jet')
line = ax[1].scatter(hit_fd[:,0], hit_fd[:,1], c=hit_idx, cmap='jet')
plt.title('Poincare hit pattern (screen)')
plt.xlabel('x [mm]')
plt.ylabel('y [mm]')
plt.colorbar(line)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [109]:
fig, (ax, bx, cx) = plt.subplots(ncols=3, figsize=(10,10/3.), sharex=True, sharey=True)
linea = ax.scatter(hit_scr[:,1]-x0[1], hit_scr[:,2]-x0[2], c=hit_idx, cmap='jet')
linec = cx.scatter(hit_scr[:,1]-x0[1], hit_scr[:,2]-x0[2], c=hit_idx, cmap='jet')
lineb = bx.scatter(hit_m[:,0], hit_m[:,1], c=hit_idx, cmap='jet')
plt.xlabel('x [mm]')
plt.ylabel('y [mm]')

#@interact(dl=(27.2, 27.9, 1e-2), ar = (0,0.1,1e-3), br = (0,0.1,1e-3), ap= (0,2*np.pi,1e-2), bp= (0,2*np.pi,1e-2), Nrt=(0,1000,100))
def update(dl=27.295, ar=0.01, br=0.0, ap=0., bp=0., Nrt=500, **kwargs):
        elements, abcd, _ = SixMirror(dx=dl)#27.2952+
        x0 = 0.5*(elements[0].p + elements[-1].p)
        n0 = norm(elements[0].p - elements[-1].p)
        screen = Screen(p=x0, n=-n0, diam=7.75)
        elements.append(screen)
        
        abcd_fd = get_ABCD_fd2(elements, x0, n0, epsr=1e-3, epss=1e-3) 
        
        system = ABCDSystem(abcd)

        mu1, mu2 = system.q
        mu = np.real(ar*np.exp(1j*ap)*mu1 + br*np.exp(1j*bp)*mu2)
        r0 = np.array([0, mu[1], mu[0]]) + x0
        a, b = mu[2:]/k
        s0 = np.array([np.sqrt(1-a**2-b**2), b, a])
        ray0 = np.stack([np.atleast_2d(r0), np.atleast_2d(s0)], axis=0)
 
        traj_hit = propagate_system_at(elements, ray0, which=6, Nrt=Nrt)[:,0,:,:] #only positions
        hit_scr = traj_hit[:,0,:]-x0
        #print(hit_scr)
        hit_idx = np.arange(hit_scr.shape[0])
        linea.set_offsets(hit_scr[:,1:])
        linea.set_array(hit_idx)
        
        
        hit_m = prop_ABCD(mu, system.abcd_rt, Nrt=Nrt)
        lineb.set_offsets(hit_m[:,0:2])
        lineb.set_array(hit_idx)
        
        hit_fd = prop_ABCD(mu, abcd_fd, Nrt=Nrt)
        linec.set_offsets(hit_fd[:,0:2])
        linec.set_array(hit_idx)
        
        ax.relim()
        ax.autoscale_view()
        fig.canvas.draw_idle()
        
interactive(update, dl=(27.2, 27.9, 1e-2), ar = (0,0.1,1e-3), br = (0,0.1,1e-3), ap= (0,2*np.pi,1e-2), bp= (0,2*np.pi,1e-2), Nrt=(0,1000,100))
#interactive(update, dl=(27.75, 27.81, 1e-3), ar = (0,0.1,1e-3), br = (0,0.1,1e-3), ap= (0,2*np.pi,1e-2), bp= (0,2*np.pi,1e-2), Nrt=(0,1000,100))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=27.295, description='dl', max=27.9, min=27.2, step=0.01), FloatSlider(…

In [ ]:
traj_eig = propagate_system(elements, reig, Nrt=1)[:,0,:,:] #only positions

In [25]:
x0

array([13.6476,  8.    ,  1.5825])

In [ ]:
#visualize perturbed mode
#visualize perturbed mode
which=0
ax=2
amp = 1e-1
dists, trajs_pert, modelmats = perturb_one_anim(SixMirror, screen, reig, traj_eig, which=which, ax=ax, rng=np.linspace(-amp, amp, 30))

In [ ]:
trajs_pert_ts = traj_to_timeseries(trajs_pert, step=1)
modelmats_ts = modelmats_to_timeseries(modelmats, step=1)

In [ ]:
plot = k3d.plot(camera_auto_fit=True, antialias=True)

for i, el in enumerate(elements[:-1]):
    pel = el.plot(opacity=0.4)
    if i==which:
        pel.color=0xff0000
    pel.model_matrix = modelmats_ts[i]
    plot += pel

#plot_trajs(traj, plot)
for ts in trajs_pert_ts:
    plot += k3d.line(ts, shader='mesh', width=0.2, color=0x00ff00)

plot.display()

In [ ]:
plot.start_auto_play()

In [ ]:
plot.stop_auto_play()

In [ ]:
#Poincare hit pattern

In [ ]:
lrs = np.zeros_like(ray0)
lrs[0,...] = 1e-3 #spatial learning rate
lrs[1,...] = 1e-1 #slope learning rate

In [ ]:
elements, _ = SixMirror(dx=27.76)

x0 = 0.5*(elements[0].p + elements[-1].p)
n0 = norm(elements[0].p - elements[-1].p)

screen = Screen(p=x0, n=-n0, diam=7.75)
elements.append(screen)

# 27.8, 0.2r0, -2e-2 s0 10 iter
#27.77 0.1 -2e-3 10

r0 = x0[None,:] + np.array([0,0.02,0])
s0 = n0[None,:] + np.array([0,-2e-4,0])
#s0 = np.broadcast_to(n0, r0.shape)

ray0 = np.stack([np.atleast_2d(r0), np.atleast_2d(s0)], axis=0)

In [ ]:
reig3, _, tols = find_eigenray_mpe_debug(elements, ray0, Nrt=3, lr=0.01, Niter=50, Nmpe=4)

In [ ]:
plt.figure()
plt.semilogy(tols)
plt.show()

In [ ]:
#traj_hit = propagate_system(elements, reig3, Nrt=2000)[:,0,:,:] #only positions
#hit_scr = traj_hit[7::7,0,:]
traj_hit = propagate_system_at(elements, reig3, which=6, Nrt=1000)[:,0,:,:] #only positions
hit_scr = traj_hit[:,0,:]
hit_idx = np.arange(hit_scr.shape[0])

plt.figure()
line = plt.scatter(hit_scr[:,1], hit_scr[:,2], c=hit_idx, cmap='jet')
plt.title('Poincare hit pattern (screen)')
plt.xlabel('x [mm]')
plt.ylabel('y [mm]')
plt.colorbar()
plt.show()

In [ ]:
fig, ax = plt.subplots()
line = plt.scatter(hit_scr[:,1], hit_scr[:,2], c=hit_idx, cmap='jet')
plt.xlabel('x [mm]')
plt.ylabel('y [mm]')

@interact(dl=(-0.1,0.1,1e-3), drx=(0,0.05,1e-3),dry=(0,0.05,1e-3),dsx=(-0.1,0.1,1e-3),dsy=(-0.1,0.1,1e-3), Nrt=(0,1000,100))
def update(dl, drx=0., dry=0., dsx=0., dsy=0., maxiter=500, Nrt=1000, **kwargs):
        elements, _ = SixMirror(dx=27.77+dl, **kwargs)

        x0 = 0.5*(elements[0].p + elements[-1].p)
        n0 = norm(elements[0].p - elements[-1].p)
        screen = Screen(p=x0, n=-n0, diam=7.75)
        elements.append(screen)
        r0 = x0[None,:] + np.array([0,drx,dry])
        s0 = n0[None,:] + np.array([0,dsx,dsy])
        ray0 = np.stack([np.atleast_2d(r0), np.atleast_2d(s0)], axis=0)
        reig3 = find_eigenray_mpe(elements, ray0, Nrt=3, lr=0.01, Niter=50, Nmpe=4)
        traj_hit = propagate_system_at(elements, reig3, which=6, Nrt=Nrt)[:,0,:,:] #only positions
        hit_scr = traj_hit[:,0,:]
        hit_idx = np.arange(hit_scr.shape[0])
  
        line.set_offsets(hit_scr[:,1:])
        line.set_array(hit_idx)
        ax.relim()
        ax.autoscale_view()
        fig.canvas.draw_idle()

In [ ]:
FFmpeg = ani.writers['ffmpeg'](fps=30, bitrate=12e3)

In [ ]:
1/30.

In [ ]:
fig, ax = plt.subplots()
line = plt.scatter(hit_scr[:,1], hit_scr[:,2], c=hit_idx, cmap='jet')
plt.xlabel('x [mm]')
plt.ylabel('y [mm]')

def init():
    line.set_offsets([[], []])
    line.set_array([])
    return line,

def animate(i, drx=0., dry=0., dsx=0.01, dsy=0., maxiter=500, Nrt=1000):
    anipar = np.linspace(-0.15, 0.15, 400)
    dl = anipar[i]
    elements, _ = SixMirror(dx=27.77+dl)
    x0 = 0.5*(elements[0].p + elements[-1].p)
    n0 = norm(elements[0].p - elements[-1].p)
    screen = Screen(p=x0, n=-n0, diam=7.75)
    elements.append(screen)
    r0 = x0[None,:] + np.array([0,drx,dry])
    s0 = n0[None,:] + np.array([0,dsx,dsy])
    ray0 = np.stack([np.atleast_2d(r0), np.atleast_2d(s0)], axis=0)
    reig3 = find_eigenray_mpe(elements, ray0, Nrt=3, lr=0.01, Niter=50, Nmpe=4)
    traj_hit = propagate_system_at(elements, reig3, which=6, Nrt=Nrt)[:,0,:,:] #only positions
    hit_scr = traj_hit[:,0,:]
    hit_idx = np.arange(hit_scr.shape[0])

    line.set_offsets(hit_scr[:,1:])
    line.set_array(hit_idx)
    #ax.relim()
    #ax.autoscale_view()
    return line,
anim = FuncAnimation(fig, animate, init_func=init, frames=400, interval=30, blit=True)
anim.save('hits2.mp4', writer=FFmpeg)
plt.show()

In [ ]:
anim = FuncAnimation(fig, animate, init_func=init, frames=200, interval=20, blit=True)

In [ ]:
init()

In [ ]:
animate(0)